In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.select import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
link_df = pd.read_parquet("./practice_youtube.parquet")
link_df.head()

,link
0,https://www.youtube.com/watch?v=yQ20jZwDjTE
1,https://www.youtube.com/watch?v=aYwg1H5BK04
2,https://www.youtube.com/watch?v=fl0BIk1e9Z0
3,https://www.youtube.com/watch?v=1b7pXC1-IbE
4,https://www.youtube.com/watch?v=hTF-z5PZ-OI


## API

In [27]:
import pandas
from googleapiclient.discovery import build

import warnings # 경고창 무시
warnings.filterwarnings('ignore')

import yaml
with open("key.yaml") as f:
    key = yaml.load(f, Loader=yaml.FullLoader)

In [29]:
YOUTUBE_API_KEY = key['YOUTUBE_API_KEY']

In [31]:
VIDEO_ID = "yQ20jZwDjTE"

comments = list()
api_obj = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)
response = api_obj.commentThreads().list(part='snippet,replies', videoId=VIDEO_ID, maxResults=100).execute()

In [32]:
while response:
    for item in response['items']:
        comment = item['snippet']['topLevelComment']['snippet']
        comments.append([comment['textDisplay'], comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])
 
        if item['snippet']['totalReplyCount'] > 0:
            for reply_item in item['replies']['comments']:
                reply = reply_item['snippet']
                comments.append([reply['textDisplay'], reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])
 
    if 'nextPageToken' in response:
        response = api_obj.commentThreads().list(part='snippet,replies', videoId='sWC-pp6CXpA', pageToken=response['nextPageToken'], maxResults=100).execute()
    else:
        break

In [33]:
df = pandas.DataFrame(comments)

In [34]:
df

,0,1,2,3
0,[웹 스크래핑 변경사항]<br>1. &quot;티스토리&quot;는 UserAgen...,나도코딩,2020-08-08T02:28:21Z,108
1,@한태웉 저도 같은 상태입니다 ㅠㅜ,김찬형,2022-12-18T11:21:21Z,0
2,나도 코딩님께! 강의 열심히 듣고 따라서 치고 배우고 있습니다. 오늘 동영상 <a ...,Jeong Hwan Lee,2022-08-23T15:08:30Z,3
3,혹시 아직 보고있으신분 계신가요 쿠팡 아예 막힌것 같아서요 cmd창에서도 pytho...,한태웉,2022-08-19T11:00:01Z,1
4,@고양이형인간 안녕하세요. 저도 같은 에러가 떠서 xpath로도 해봤는데 진행이 ...,Sungeun,2022-03-14T07:34:12Z,0
...,...,...,...,...
506,3,가나다,2022-01-12T11:11:52Z,0
507,1,펭구,2022-01-12T11:11:49Z,0
508,1등이네 ㅎㅎ,애플은사과,2022-01-12T11:11:47Z,0
509,오,테르밋,2022-01-12T11:11:44Z,0


## SELENIUM

In [13]:
List = []
url_list = link_df.link.to_list()
driver = webdriver.Chrome("C:/Workspace/chromedriver.exe")

for url in tqdm(url_list[14:]):
    driver.get(url)
    driver.implicitly_wait(10)
    driver.find_element_by_css_selector("body").send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    comment_height = last_height
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    while True:
        for _ in range(5):
            driver.find_element_by_css_selector("body").send_keys(Keys.PAGE_DOWN)
        driver.implicitly_wait(5)
        time.sleep(3)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    driver.execute_script(f"window.scrollTo(0, {comment_height});")
    time.sleep(1.5)
    comment_elements = driver.find_elements_by_css_selector('#more-replies > yt-button-shape > button > yt-touch-feedback-shape > div > div.yt-spec-touch-feedback-shape__fill')
    
    for element in comment_elements:
        try:
            driver.find_element_by_xpath('//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
        except:
            pass
        try:
            element.click()
        except:
            continue
        
    more_elements = driver.find_elements_by_css_selector('#more > span')
    driver.execute_script(f"window.scrollTo(0, {comment_height});")
    for element in more_elements:
        # try:
        #     driver.find_element_by_xpath('//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
        # except:
        #     pass
        if element.text == "자세히 보기":
            try:
                element.click()
            except:
                continue
    comment = driver.find_elements_by_css_selector('#content-text')
    for text in comment:
        List.append(text.text)
driver.quit()
print(len(List))

df = pd.DataFrame(List, columns=["comment"])
df.to_parquet("./comment6.parquet")

100%|██████████| 652/652 [11:05:22<00:00, 61.23s/it]    


17323


In [11]:
comment = driver.find_elements_by_css_selector('#content-text')
for text in tqdm(comment):
    List.append(text.text)  
driver.quit()
print(len(List))

100%|██████████| 34/34 [00:03<00:00,  8.60it/s]


205


In [2]:
df = pd.DataFrame(columns=["comment"])
for num in range(1,7):
    df = pd.concat([df,pd.read_parquet(f"./comment{num}.parquet")])

In [4]:
df.info()
df.to_parquet("./data.parquet")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21187 entries, 0 to 17322
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  21187 non-null  object
dtypes: object(1)
memory usage: 331.0+ KB


In [5]:
pd.read_parquet("./practice.parquet")

,code,title,rate,review,price_range,days
0,436639,"일회성 크롤링, 웹 크롤링, 웹 매크로 프로그램 제작",5.0,신속하게 작업 진행해주었고 결과물도 만족스럽습니다.,10만원 ~ 20만원,1일
1,436639,"일회성 크롤링, 웹 크롤링, 웹 매크로 프로그램 제작",5.0,전번에 의뢰드렸던 프로그램이 너무좋아서 다른개발건으로 의뢰 드렸는데 너무좋습니다! ...,20만원 ~ 30만원,3일
2,436639,"일회성 크롤링, 웹 크롤링, 웹 매크로 프로그램 제작",5.0,너무 친절하시고 실력 좋으십니다. 강력 추천합니다!,10만원 ~ 20만원,4일
3,436639,"일회성 크롤링, 웹 크롤링, 웹 매크로 프로그램 제작",5.0,대표님 이해도가 정말 빠르셔서 수월했습니다 완벽하게 만들어주셔서 감사합니다 여기는 ...,10만원 ~ 20만원,24시간이내
4,436639,"일회성 크롤링, 웹 크롤링, 웹 매크로 프로그램 제작",5.0,완벽한 결과물을 만들어주셨습니다. 진행중인 프로젝트가 계속 있는데 개발자님과 계속 ...,20만원 ~ 30만원,6일
...,...,...,...,...,...,...
5437,407181,티스토리 블로그 기본 세팅 및 SEO 최적화,5.0,"선생님 스킬은 넘사벽입니다. 정확하게 네이버,구글,빙 검색엔진에 탑재해 주십니다....",5만원 미만,24시간이내
5438,407181,티스토리 블로그 기본 세팅 및 SEO 최적화,5.0,너무 감사드립니다. 저같은 초보분들은 꼭 받아 보시기 바랍니다. 선생님 너무 감사 ...,5만원 미만,24시간이내
5439,407181,티스토리 블로그 기본 세팅 및 SEO 최적화,5.0,신속하고 빠르게 봐주시고 좋아요. 믿고 맡길 수 있습니다~,5만원 미만,24시간이내
5440,407181,티스토리 블로그 기본 세팅 및 SEO 최적화,5.0,너무 상세하게 해주셔서 감사합니다. 가끔씩. 문의하겠습니다.,5만원 미만,24시간이내


In [8]:
df

,comment
0,"[웹 스크래핑 변경사항]\n1. ""티스토리""는 UserAgent 를 변경하지 않아도..."
1,@1814배종환 0:02:03 부터 설명하는 내용 봐주시면 됩니다 ^^
2,파이썬으로 모바일 앱도 만들수 있나요?
3,kivy 를 이용하시면 가능합니다만 파이썬으로 앱개발을 하는 것은 추천드리지 않습니...
4,@나도코딩 앱개발에는 무슨 언어를 추천하시나요?
...,...
17318,마싰겠드앙
17319,2등?
17320,노렁ㅅ
17321,단맛없는 액상 오랜만이네요


In [11]:
for i in more_elements[2:]:
    try:
        driver.find_element_by_xpath('//*[@id="dismiss-button"]/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]').click()
    except:
        pass
    if i.text == "자세히 보기":
        i.click()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.105)


In [9]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    
    driver.find_element_by_css_selector("body").send_keys(Keys.PAGE_DOWN)
    driver.implicitly_wait(5)
    time.sleep(2)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

In [21]:
comment_elements = driver.find_elements_by_css_selector('#more-replies > yt-button-shape > button > yt-touch-feedback-shape > div > div.yt-spec-touch-feedback-shape__fill')
more_elements = driver.find_elements_by_css_selector('#more > span')
len(comment_elements), len(more_elements)

(25, 69)

In [22]:
for element in more_elements:
    if element.text == "자세히 보기":
        element.click()

In [23]:
List = []
comment = driver.find_elements_by_css_selector('#content-text')
for text in comment:
    List.append(text.text)
#content-text
len(List)

70

In [24]:
List

['단순하고 명쾌하네요. 초보자들이 이해하기 쉽습니다.',
 '아~ 그러시면 너무 좋네요.',
 '쉽게 잘 설명해주셔서 감사합니다 ^^',
 'Javascript하고 문법이 비슷하네요. 관심있는 문서를 크롤링 하는방법을 찾다가 이런방법이 있구나 해서 봤는데 오늘 해봐야겠습니다.',
 '안녕하세요. 화학물질 정보를 엑셀로 구축하고 싶은데요, 예를들어 물질명을 알면 그에 해당하는 CAS #, chemical formular 등을 특정 웹에서 추출하여 엑셀로 빠르게 정리하고 싶은데 이런 코딩은 웹 스크래핑으로 가능한건가요? 가능하다면 기술노트님의 영상 중 어떤거를 보면 좋을지도 추천 부탁드립니다!',
 '안녕하세요~ 혹시 파이썬으로 만드실껀가요? 그리고 해당 사이트가 있다라는 말씀이시죠? 그 사이트에 가면 물질명으로 정보를 찾을 수가 있는 것이고, 그것을 가져오고 싶다는 얘기인거 같은데요. 혹시 지금 보신 영상을 활용하시면 어려움이 있으시나요? 엑셀로 저장하는 것은 별도의 라이브러리들이 있구요. 사이트에서 가져오는 것은 기본 로직이 비슷해서요. 해당 사이트의 상황으로 바꿔주시면 될거 같은데요. 해보시고 안되시면 질문주시구요~',
 '말씀하신 것도 현재 공유드린 크롤링 기술을 응용하면 가능할거 같은데요. 그런데, 특정 사이트에 그 내용들이 좀 있어야 하구요. 검색을 해서 어느 사이트든 들어가서 가져오는 것은 쉽지는 않을거 같네요. 쉽게 말해 해당 내용이 모아져 있는 사이트가 있으면 규칙에 따라 가져 올 수 있을거 같아요.',
 '크롤링으로 블로그에 작성된 1000개정도 되는글을 폴더화시킨다고 했을때 사진은jpg으로 순서에맞게 이름변경돼서 폴더에 저장, 글원고의 제목은 블로그글의제목으로 글내용은 한글이나 워드로 저장하는 작업도 가능한건가요?',
 '크롤링을 사용하여 블로그의 내용을 가져오는 것은 가능하고, 이미지 저장도 가능합니다. 다만 한글이나 워드로 저장하는 부분이 라이브러리가 필요할 거 같은데요. 대개는 크롤링한 데이터를 TXT나 CSV정도로 저장을 하거든요. 그